In [ ]:
import numpy as np
from matplotlib import pyplot
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping
import matplotlib.pyplot as plt
from matplotlib import gridspec
import random
import imageio
import os
import cv2
import glob
import tensorflow as tf
random.seed(42)

import wandb
from wandb.keras import WandbCallback

# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip -q '/content/nature_12K.zip'

imgsize = 128


# Load training and testing dataset
def TestingData(DataAugumentationFlag, batch_size):
  if DataAugumentationFlag:
      datagen = ImageDataGenerator(
                rotation_range=45, 
                width_shift_range=0.2, 
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2, 
                horizontal_flip=True, 
                fill_mode='reflect',
                rescale=1./255
                )
  else:
      datagen = ImageDataGenerator(rescale=1./255)

  TrainData = datagen.flow_from_directory(
              directory='/content/inaturalist_12K/train',
              target_size = (imgsize,imgsize),
              batch_size=batch_size,
              class_mode="categorical",
              shuffle=True,
              seed=22)

  test_data = datagen.flow_from_directory(
            directory='/content/inaturalist_12K/val',
            target_size=(imgsize,imgsize),
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=22)
  return test_data, TrainData


In [ ]:
def test():
    KernalSize = [(3,3),(3,3),(3,3),(3,3),(3,3)]
    WeightDecay = 0
    Dropout = 0.2
    lr = 1e-4
    activation = 'selu'
    bs = 64
    batchnorm = 'false'
    filterorg = [32,64,64,128,128]
    dataagumentationflag = 'true'
    denselayer = 256
    input_shape = (imgsize, imgsize, 3)
    model = Sequential()
    filter = filterorg
    model.add(Conv2D(filters = filter[0], kernel_size = KernalSize[0],padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(WeightDecay)))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(filters = filter[1], kernel_size = KernalSize[1],padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(WeightDecay)))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(filters = filter[2], kernel_size = KernalSize[2],padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(WeightDecay)))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters = filter[3], kernel_size = KernalSize[3],padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(WeightDecay)))
    model.add(Activation('elu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters = filter[4], kernel_size = KernalSize[4],padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(WeightDecay)))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(denselayer, activation = activation, kernel_regularizer = regularizers.l2(WeightDecay)))
    model.add(Dropout(Dropout))

    model.add(Dense(10, activation = "softmax"))

    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=['accuracy'])
    TrainData, test_data = TestingData(dataagumentationflag, bs)
    model.fit(TrainData, epochs=1)
    model.evaluate(test_data, batch_size = bs)
    model.save("model-best.h5")
    return model

model = test()

In [ ]:

class_names = {0:'Amphibia', 1:'Animalia', 2:'Arachnida',3: 'Aves',4: 'Fungi',
              5: 'Insecta', 6:'Mammalia', 7:'Mollusca', 8:'Plantae',9: 'Reptilia'}

img_size = imgsize
dir = '/content/inaturalist_12K/val'

visuvalizationtestimages = []
visuvalizationtruelabels = []


for label, name in class_names.items():
  list_images = os.listdir(dir+'/'+name)
  res = []
  for j in range(3):
    dummy = random.randint(0, 199)
    res.append(list_images[dummy])
  for image_name in res:
      image = imageio.imread(dir+'/'+name+'/'+image_name)
      if np.ndim(image) == 3:
        visuvalizationtestimages.append(cv2.resize(image, (img_size,img_size)))
        visuvalizationtruelabels.append(class_names[label])

visuvalizationtestimages = np.array(visuvalizationtestimages)
visuvalizationtestimages = visuvalizationtestimages/255.0
fig = plt.figure(figsize=(30,20))

gs = gridspec.GridSpec(10, 3, width_ratios=[1, 1, 1], wspace=0.0, hspace=1.0, top=0.95, bottom=0.05, left=0.7, right=0.845) 
k = 0
for i in range(10):
    for j in range(3):
      ax = plt.subplot(gs[i,j])
      img = visuvalizationtestimages[k]
      ax.imshow(img)
      prediction = model.predict(img.reshape(1,imgsize, imgsize, 3))[0]
      true_label = visuvalizationtruelabels[k]
      k = k + 1
      ax.axis('off')
      ax.set_title("True Label: " + true_label + "\nPredicted: " + class_names[np.argmax(prediction)])

plt.show()
fig.savefig('predictions.png')


In [ ]:

for layer in model.layers:
	if 'conv' not in layer.name:
		continue
	filters, biases = layer.get_weights()
	print(layer.name, filters.shape)

filters, biases = model.layers[0].get_weights()
f_minimum, f_maximum = filters.min(), filters.max()
filters = (filters - f_minimum) / (f_maximum - f_minimum)

fig = plt.figure(figsize=(imgsize,imgsize))
gs = gridspec.GridSpec(32, 3, width_ratios=[1, 1, 1], wspace=0.0, hspace=0.5, top=0.95, bottom=0.05, left=0.7, right=0.845) 
n_filters, ix = 32, 1

for i in range(n_filters):    #32
    f = filters[:, :, :, i]
    for j in range(3):        #3 
      ax = plt.subplot(gs[i,j])
      ax.imshow(f[:, :, j], cmap='gray')
      ax.set_xticklabels([])
      ax.set_yticklabels([])

plt.show()
fig.savefig('layersconvolutional.png')

In [ ]:

imageindex = 1
image = visuvalizationtestimages[imageindex]
img_test = np.expand_dims(image, axis=0)

refinedneuralnet = tf.keras.models.Model([model.inputs],[model.get_layer("conv2d_4").output])



image = visuvalizationtestimages[imageindex]
img_test = np.expand_dims(image, axis=0)

plt.imshow(img_test[0])
plt.title("True Label: " + visuvalizationtruelabels[imageindex])
plt.axis("off")
plt.show()

refinedneuralnet = tf.keras.models.Model([model.inputs],[model.get_layer("conv2d_4").output])

@tf.custom_gradient
def guidedRelU(x):
  def grad(dy):
    return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
  return tf.nn.relu(x), grad

for layer in model.layers[1:]:
    if hasattr(layer, 'activation') and layer.activation == tf.keras.activations.selu:
        layer.activation = guidedRelU


In [ ]:
with tf.GradientTape() as g:
  inputs = tf.cast(img_test, tf.float32)
  g.watch(inputs)
  outputs = refinedneuralnet(inputs)[0]

target_gradient = g.gradient(outputs,inputs)[0]

CONV5_layer_activation = outputs
layer_names = ['conv2d_4']
num_features = 10
size = 1
num_imag_in_row = 1
num_cols = num_features // num_imag_in_row
gridofimages = np.ones((size * num_cols, num_imag_in_row * size))

k = 1
j = 1
for col in range(num_cols): 
  for row in range(num_imag_in_row):
    channel_image = CONV5_layer_activation[j, k, col * num_imag_in_row + row]
    gridofimages[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
  
scale = 1. / size
plt.figure(figsize=(scale * gridofimages.shape[1], scale * gridofimages.shape[0]))
plt.title(layer_names)
plt.grid(False)
plt.imshow(gridofimages, aspect='auto',cmap='gray')  
plt.show()                                         
fig.savefig('neuronfire.png')

gradientimage = np.dstack((
            target_gradient[:, :, 0],
            target_gradient[:, :, 1],
            target_gradient[:, :, 2],
        ))       

gradientimage = gradientimage - np.min(gradientimage)
gradientimage = gradientimage/gradientimage.max()
imgplot = plt.imshow(gradientimage)
plt.axis("off")
plt.show()
fig.savefig('gradients.png')



In [ ]:


# The shape of the layer that we are interested in
conv_output_shape = model.layers[-8].output.shape[1:]

plt.figure(figsize=(30, 60))
for i in range(10):
    # Index of a random pixel
    Nx = np.random.randint(0, conv_output_shape[0])
    Ny = np.random.randint(0, conv_output_shape[1])
    Nz = np.random.randint(0, conv_output_shape[2])

    # Mask to focus on the outputs of only one neuron in the last convolution layer
    mat = np.zeros((1, *conv_output_shape), dtype="float")
    mat[0, Nx, Ny, Nz] = 1

    # Calculate the gradients
    with tf.GradientTape() as tape:
        inputs = tf.cast(np.array([np.array(img)]), tf.float32)
        tape.watch(inputs)
        outputs = refinedneuralnet(inputs) * mat

    grads = tape.gradient(outputs, inputs)[0]

    # Visualize the output of guided backpropagation
    imgguidedbp = np.dstack((grads[:, :, 0], grads[:, :, 1], grads[:, :, 2],)) 

    # Scaling to 0-1      
    imgguidedbp = imgguidedbp - np.min(imgguidedbp)
    imgguidedbp /= imgguidedbp.max()
    plt.subplot(10, 1, i+1)
    plt.imshow(imgguidedbp)
    plt.axis("off")

plt.show()